In [113]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, max as max_
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

In [114]:
spark = SparkSession.builder.appName('Salaries Differences').getOrCreate()

In [115]:
df_employee_input= spark.read.options(header= 'True', InferSchema='True').csv('db_employee.csv')
df_dept_input= spark.read.options(header= 'True', InferSchema='True').csv('db_dept.csv')
#df_employee_input.show()
#df_dept_input.show()

In [116]:
df_grouped_salary = df_employee_input.groupBy('department_id').max('salary').orderBy('department_id').select(col("department_id"),col("max(salary)").alias("max_salary"))

In [117]:
df_joined = df_grouped_salary.join(df_dept_input, df_grouped_salary.department_id == df_dept_input.id, 'inner').orderBy('department_id').selectExpr('department_id','max_salary','department')

In [118]:
df_joined.createOrReplaceTempView("dept")
df_dept_filtered= spark.sql("select a.department_id,a.max_salary from dept a where a.department in('marketing','engineering')")

In [121]:
window = Window.partitionBy().orderBy("department_id")
df_output = df_dept_filtered.withColumn("salary_difference", col("max_salary") - F.lag(col("max_salary"), 1).over(window))

In [122]:
df_output.filter("salary_difference IS NOT NULL").selectExpr('salary_difference').show()

+-----------------+
|salary_difference|
+-----------------+
|             2400|
+-----------------+

